In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import keras

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import data_prepare

Using TensorFlow backend.


In [3]:
post, thread=data_prepare.load_train_data()
post_test, thread_test=data_prepare.load_test_data()
label_map=data_prepare.load_label_map()

In [135]:
num=len(thread)
train_data_to_clean=[]
test_data_to_clean=[]

#train_data_to_clean=data_prepare.get_all_text_data_from_posts(post, thread)
#test_data_to_clean=data_prepare.get_all_text_data_from_posts(post_test, thread_test)


clean_train_data = [data_prepare.clean(s) for s in thread["thread_name"]]
clean_test_data = [data_prepare.clean(s) for s in thread["thread_name"]]

In [174]:
import warnings
warnings.filterwarnings('ignore')


X_test=clean_test_data
X_train=clean_train_data
y_train=thread["thread_label_id"]
#X_train, X_val, y_train, y_val = train_test_split(clean_train_data, thread["thread_label_id"], test_size=0.2, random_state=213)

In [175]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=3,sublinear_tf=True, norm='l2', encoding='latin-1', ngram_range=(1, 1))

train_data_features = vectorizer.fit_transform(X_train).toarray()

test_data_features = vectorizer.transform(X_test).toarray()

In [176]:
def get_unique_users(post):
    return post.groupby(['thread_num','user'],sort=False).size().reset_index('user').rename(index=str, columns={0: "number_of_posts"})
def avg_and_dev_in_posts_number(df):
    return df.groupby("thread_num",sort=False).mean(), df.groupby("thread_num",sort=False).std()
def normalize(s):
    return (s-s.mean())/s.max()




def get_quotes_data(post):
    quotes_data=post[["thread_num","quotes"]]
    a=quotes_data.groupby("thread_num",sort=False).size()
    b=quotes_data[quotes_data["quotes"]!=''].groupby("thread_num",sort=False).size()
    numb_quotes=pd.Series(index=a.index)
    for index,item in a.items():
        if index in b.index:
            numb_quotes[index]=b.loc[index]
        else:
            numb_quotes[index]=0
    q_perc=(numb_quotes/a).as_matrix()
    numb_quotes=normalize(numb_quotes.astype(int).as_matrix())

    return numb_quotes, q_perc

In [177]:
unique_users = get_unique_users(post)
numb_unique=unique_users.groupby("thread_num",sort=False).nunique()["user"]
numb_unique= normalize(numb_unique)

avg_numb_of_posts, dev_numb_of_posts = avg_and_dev_in_posts_number(unique_users[["number_of_posts"]])
numb_quotes, q_perc=get_quotes_data(post)

In [178]:
unique_users_test = get_unique_users(post_test)
numb_unique_test=unique_users_test.groupby("thread_num",sort=False).nunique()["user"]
numb_unique_test=normalize(numb_unique_test)

avg_numb_of_posts_test, dev_numb_of_posts_test = avg_and_dev_in_posts_number(unique_users_test[["number_of_posts"]])
numb_quotes_test, q_perc_test=get_quotes_data(post_test)

In [179]:
number_train_posts=thread["thread_replies"].as_matrix()
number_train_posts=normalize(number_train_posts)

number_test_posts=thread_test["thread_replies"].as_matrix()
number_test_posts=normalize(number_test_posts)


train_data_features=np.concatenate([number_train_posts.reshape(-1,1), numb_unique.as_matrix().reshape(-1,1),
                                     numb_quotes.reshape(-1,1),q_perc.reshape(-1,1),
                                     avg_numb_of_posts.as_matrix().reshape(-1,1),train_data_features],axis=1)
test_data_features=np.concatenate([number_test_posts.reshape(-1,1),numb_unique_test.as_matrix().reshape(-1,1),
                             numb_quotes_test.reshape(-1,1),q_perc_test.reshape(-1,1),
                             avg_numb_of_posts_test.as_matrix().reshape(-1,1),test_data_features],axis=1)

In [184]:
train_X=np.delete(train_data_features,np.s_[101],axis=1)
train_X=train_X.reshape(358,10,10,1)

(358, 140293) (236, 140293)


In [96]:
classes=np.unique(y_train)
classes

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [97]:
from keras.utils import to_categorical
train_Y_one_hot = to_categorical(y_train)

In [185]:
from sklearn.model_selection import train_test_split
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.1, random_state=13)

In [131]:
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [191]:
batch_size = 10
epochs = 20
num_classes = 13

In [186]:
mafia_model = Sequential()
mafia_model.add(Conv2D(16, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(374,374,1)))
mafia_model.add(LeakyReLU(alpha=0.1))
mafia_model.add(MaxPooling2D((2, 2),padding='same'))
mafia_model.add(Dropout(0.25))
mafia_model.add(Conv2D(32, (3, 3), activation='linear',padding='same'))
mafia_model.add(LeakyReLU(alpha=0.1))
mafia_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
mafia_model.add(Dropout(0.25))
mafia_model.add(Flatten())
mafia_model.add(Dense(32, activation='linear'))
mafia_model.add(LeakyReLU(alpha=0.1))           
mafia_model.add(Dropout(0.3))
mafia_model.add(Dense(num_classes, activation='softmax'))

In [132]:
model = Sequential()
model.add(Conv1D(101, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(101, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(101, 5, activation='relu'))
model.add(MaxPooling1D(35))
model.add(Flatten())
model.add(Dense(101, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [187]:
mafia_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [188]:
mafia_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 374, 374, 16)      160       
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 374, 374, 16)      0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 187, 187, 16)      0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 187, 187, 16)      0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 187, 187, 32)      4640      
_________________________________________________________________
leaky_re_lu_35 (LeakyReLU)   (None, 187, 187, 32)      0         
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 94, 94, 32)        0         
__________

In [192]:
mafia_train = mafia_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

Train on 322 samples, validate on 36 samples
Epoch 1/20
322/322 [==============================] - 89s 277ms/step - loss: 1.5590 - acc: 0.5559 - val_loss: 1.3564 - val_acc: 0.6389
Epoch 2/20
322/322 [==============================] - 81s 252ms/step - loss: 1.4038 - acc: 0.5745 - val_loss: 1.2290 - val_acc: 0.6111
Epoch 3/20
322/322 [==============================] - 80s 248ms/step - loss: 1.4109 - acc: 0.5466 - val_loss: 0.9746 - val_acc: 0.6389
Epoch 4/20
322/322 [==============================] - 80s 248ms/step - loss: 1.4214 - acc: 0.5528 - val_loss: 1.0361 - val_acc: 0.6389
Epoch 5/20
322/322 [==============================] - 88s 274ms/step - loss: 1.3107 - acc: 0.5776 - val_loss: 1.0807 - val_acc: 0.6389
Epoch 6/20
322/322 [==============================] - 82s 254ms/step - loss: 1.1942 - acc: 0.6584 - val_loss: 0.9193 - val_acc: 0.6111
Epoch 7/20
322/322 [==============================] - 82s 256ms/step - loss: 0.9739 - acc: 0.6708 - val_loss: 0.7103 - val_acc: 0.8333
Epoch 8/20

In [193]:
predicted_classes = mafia_model.predict(test_X)

In [194]:
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

In [214]:
lab=pd.Series(pred)
ans = pd.concat([thread_test["thread_num"],lab], axis=1, keys=['thread_num', 'thread_label_id'])
ans=ans.set_index("thread_num")
ans.head()

,thread_label_id
thread_num,
126856,8
132415,8
134482,0
133728,1
134270,8


In [215]:
path=os.path.join(module_path,"submissions")
ans.to_csv(os.path.join(path,"sol29.csv"))